In [13]:
import os
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import plotly.express as px
import plotly
import plotly.graph_objects as go

In [14]:
scenarios = ['Reference', 'Improve AG eff', 'New Resources', 'Reduce NRW']
# scenarios = ['Reference']
df_all = {}
for i in ['Pipelines_data','Water_requirements','Water_delivered',
                         'Groundwater_pumping','wwtp_data','desal_data']:
    df_all[i] = pd.DataFrame()
    
load_folder = os.path.join('dashboard', 'data')
for scenario in scenarios:
    scenario_folder = os.path.join(load_folder, scenario)
    for sub_scenario in os.listdir(scenario_folder):
        sub_scenario_folder = os.path.join(scenario_folder, sub_scenario)
        for level in os.listdir(sub_scenario_folder):
            load_data = os.path.join(sub_scenario_folder, level)
            for file in ['Pipelines_data','Water_requirements','Water_delivered',
                         'Groundwater_pumping','wwtp_data','desal_data']:
                file_path = os.path.join(load_data, file + '.csv')
                _df = pd.read_csv(file_path)
                _df['scenario'] = scenario
                _df['subscenario'] = sub_scenario
                if scenario == 'Improve AG eff':
                    if level == 'level_1':
                        _level = 'by 10%'
                    else:
                        _level = 'by 20%'
                elif scenario == 'Reduce NRW':
                    if level == 'level_1':
                        _level = 'to 40%'
                    else:
                        _level = 'to 20%'
                else:
                    _level = ''
                _df['level'] = _level
                _df['name'] = f"{scenario} {_level}"
                df_all[file] = df_all[file].append(_df, ignore_index=True, sort=False)
                

In [15]:
dff = df_all['Water_delivered']
dff = dff.groupby(['subscenario', 'scenario','type','Year']).sum()/1000000
dff.reset_index(inplace=True)
dff = dff.loc[dff.scenario=='Reference']
fig = px.area(dff, x="Year", y="value", color='type', facet_col='subscenario')
fig.update_layout({'legend_orientation':'h'})
fig.update_layout({'title':'Water delivered Reference scenario','yaxis_title':'Water demand (Mm3)'})
fig.update_layout(legend=dict(x=0, y=-0.2))
fig.show()
# plotly.offline.plot(fig, filename='name.html')
fig.write_image('Reference_water_delivered.png', width=700, height=400, scale=4)

In [16]:
dff = pd.DataFrame()
df_all['Pipelines_data']['type'] = 'Water conveyance'
for df in ['Pipelines_data','Groundwater_pumping','wwtp_data','desal_data']:
    dff = dff.append(df_all[df], ignore_index=True, sort=False)
dff = dff.groupby(['subscenario', 'scenario','type','Year']).sum()/1000000/0.45
dff.reset_index(inplace=True)
dff = dff.loc[dff.scenario=='Reference']
fig = px.area(dff, x="Year", y="SWPA_E_", color='type', facet_col='subscenario')
fig.update_layout({'legend_orientation':'h'})
fig.update_layout({'title':'Energy demand Reference scenario','yaxis_title':'Energy demand (GWh)'})
fig.update_layout(legend=dict(x=0, y=-0.2))
fig.show()
# plotly.offline.plot(fig, filename='name.html')
fig.write_image('Reference_energy.png', width=700, height=400, scale=4)

In [17]:
dff = df_all['Water_requirements']
dff_2 = df_all['Water_delivered']
dff = dff.groupby(['subscenario', 'scenario','type','Year']).sum()/1000000
dff_2 = dff_2.groupby(['subscenario', 'scenario','type','Year']).sum()/1000000

dff['water_delivered'] = dff.index.map(dff_2.value)

dff.reset_index(inplace=True)

dff['unmet_demand'] = (dff.value - dff.water_delivered) / dff.value

dff = dff.loc[dff.scenario=='Reference']

fig = px.line(dff, x="Year", y="unmet_demand", color='type', facet_col='subscenario')
fig.update_layout({'legend_orientation':'h'})
fig.update_layout({'title':'Unmet water demand Reference scenario','yaxis_title':'percentage (%)'})
fig.update_layout(legend=dict(x=0, y=-0.2))
fig.show()
fig.write_image('Unmet_demand.png', width=700, height=400, scale=4)

In [18]:
dff = df_all['Water_requirements']
dff_2 = df_all['Water_delivered']
dff = dff.groupby(['subscenario', 'scenario','level','type','Year']).sum()/1000000
dff_2 = dff_2.groupby(['subscenario', 'scenario','level','type','Year']).sum()/1000000

dff['water_delivered'] = dff.index.map(dff_2.value)

dff.reset_index(inplace=True)

dff['unmet_demand'] = (dff.value - dff.water_delivered) / dff.value

dff.reset_index(inplace=True)
dff = dff.loc[dff.scenario.isin(['Reference','Improve AG eff']) & (dff.subscenario=='Eto trend') & ((dff.level=='by 20%') | (dff.level==''))]
fig = px.line(dff, x="Year", y="unmet_demand", color='scenario', facet_col='type')
fig.update_layout({'legend_orientation':'h'})
fig.update_layout({'title':'Unmet water demand Improve AG eff by 20% scenario','yaxis_title':'percentage (%)'})
fig.update_layout(legend=dict(x=0, y=-0.2))
fig.show()
# plotly.offline.plot(fig, filename='name.html')
fig.write_image('Improve_AG_unmet.png', width=900, height=400, scale=4)

In [19]:
dff = pd.DataFrame()
df_all['Pipelines_data']['type'] = 'Water conveyance'
for df in ['Pipelines_data','Groundwater_pumping','wwtp_data','desal_data']:
    dff = dff.append(df_all[df], ignore_index=True, sort=False)
dff = dff.groupby(['subscenario', 'scenario','level','type','Year']).sum()/1000000/0.45
dff.reset_index(inplace=True)
dff = dff.loc[dff.scenario.isin(['Reference','Improve AG eff']) & (dff.subscenario=='Eto trend') & ((dff.level=='by 20%') | (dff.level==''))]
fig = px.line(dff, x="Year", y="SWPA_E_", color='scenario', facet_col='type')
fig.update_layout({'legend_orientation':'h'})
fig.update_layout({'title':'Energy demand Improve AG eff by 20% scenario','yaxis_title':'Energy demand (GWh)'})
fig.update_layout(legend=dict(x=0, y=-0.2))
fig.show()
# plotly.offline.plot(fig, filename='name.html')
fig.write_image('Improve_AG_energy.png', width=1000, height=400, scale=4)

In [20]:
dff = df_all['Water_requirements']
dff_2 = df_all['Water_delivered']
dff = dff.groupby(['subscenario', 'scenario','level','type','Year']).sum()/1000000
dff_2 = dff_2.groupby(['subscenario', 'scenario','level','type','Year']).sum()/1000000

dff['water_delivered'] = dff.index.map(dff_2.value)

dff.reset_index(inplace=True)

dff['unmet_demand'] = (dff.value - dff.water_delivered) / dff.value

dff.reset_index(inplace=True)
dff = dff.loc[dff.scenario.isin(['Reference','Reduce NRW']) & (dff.subscenario=='Eto trend') & ((dff.level=='to 20%') | (dff.level==''))]
fig = px.line(dff, x="Year", y="unmet_demand", color='scenario', facet_col='type')
fig.update_layout({'legend_orientation':'h'})
fig.update_layout({'title':'Unmet water demand Reduce NRW to 20% scenario','yaxis_title':'percentage (%)'})
fig.update_layout(legend=dict(x=0, y=-0.2))
fig.show()
# plotly.offline.plot(fig, filename='name.html')
fig.write_image('Reduce_NRW_unmet.png', width=900, height=400, scale=4)

In [21]:
dff = pd.DataFrame()
df_all['Pipelines_data']['type'] = 'Water conveyance'
for df in ['Pipelines_data','Groundwater_pumping','wwtp_data','desal_data']:
    dff = dff.append(df_all[df], ignore_index=True, sort=False)
dff = dff.groupby(['subscenario', 'scenario','level','type','Year']).sum()/1000000/0.45
dff.reset_index(inplace=True)
dff = dff.loc[dff.scenario.isin(['Reference','Reduce NRW']) & (dff.subscenario=='Eto trend') & ((dff.level=='to 20%') | (dff.level==''))]
fig = px.line(dff, x="Year", y="SWPA_E_", color='scenario', facet_col='type')
fig.update_layout({'legend_orientation':'h'})
fig.update_layout({'title':'Energy demand Reduce NRW to 20% scenario','yaxis_title':'Energy demand (GWh)'})
fig.update_layout(legend=dict(x=0, y=-0.2))
fig.show()
# plotly.offline.plot(fig, filename='name.html')
fig.write_image('Reduce_NRW_energy.png', width=1000, height=400, scale=4)

In [22]:
dff = df_all['Water_requirements']
dff_2 = df_all['Water_delivered']
dff = dff.groupby(['subscenario', 'scenario','level','type','Year']).sum()/1000000
dff_2 = dff_2.groupby(['subscenario', 'scenario','level','type','Year']).sum()/1000000

dff['water_delivered'] = dff.index.map(dff_2.value)

dff.reset_index(inplace=True)

dff['unmet_demand'] = (dff.value - dff.water_delivered) / dff.value

dff.reset_index(inplace=True)
dff = dff.loc[dff.scenario.isin(['Reference','New Resources']) & (dff.subscenario=='Eto trend') & (dff.level=='')]
fig = px.line(dff, x="Year", y="unmet_demand", color='scenario', facet_col='type')
fig.update_layout({'legend_orientation':'h'})
fig.update_layout({'title':'Unmet water demand New Resources scenario','yaxis_title':'percentage (%)'})
fig.update_layout(legend=dict(x=0, y=-0.2))
fig.show()
# plotly.offline.plot(fig, filename='name.html')
fig.write_image('New_Resources_unmet.png', width=900, height=400, scale=4)

In [23]:
dff = pd.DataFrame()
df_all['Pipelines_data']['type'] = 'Water conveyance'
for df in ['Pipelines_data','Groundwater_pumping','wwtp_data','desal_data']:
    dff = dff.append(df_all[df], ignore_index=True, sort=False)
dff = dff.groupby(['subscenario', 'scenario','level','type','Year']).sum()/1000000/0.45
dff.reset_index(inplace=True)
dff = dff.loc[dff.scenario.isin(['Reference','New Resources']) & (dff.subscenario=='Eto trend') & ((dff.level=='to 20%') | (dff.level==''))]
fig = px.line(dff, x="Year", y="SWPA_E_", color='scenario', facet_col='type')
fig.update_layout({'legend_orientation':'h'})
fig.update_layout({'title':'Energy demand New Resources scenario','yaxis_title':'Energy demand (GWh)'})
fig.update_layout(legend=dict(x=0, y=-0.2))
fig.show()
# plotly.offline.plot(fig, filename='name.html')
fig.write_image('New_Resources_energy.png', width=1000, height=400, scale=4)

In [24]:
dff = df_all['Water_requirements']
dff_2 = df_all['Water_delivered']
dff = dff.groupby(['subscenario', 'name','Year']).sum()/1000000
dff_2 = dff_2.groupby(['subscenario', 'name','Year']).sum()/1000000

dff['water_delivered'] = dff.index.map(dff_2.value)

dff.reset_index(inplace=True)

dff['unmet_demand'] = (dff.value - dff.water_delivered) / dff.value

fig = px.line(dff, x="Year", y="unmet_demand", color='name', facet_col='subscenario')
fig.update_layout({'legend_orientation':'h'})
fig.show()